In [1]:
from importlib import reload

import pandas as pd

import src.severa.base_client
import src.severa.fetch
from src.daterange import DateRange
reload(src.severa.base_client)
reload(src.severa.fetch)

span = DateRange(540)

f = src.severa.fetch.Fetcher()
async with f._client:
    df = await f.get_resource_allocations(span)

2023-05-30 15:59:32.956 | INFO     | src.severa.fetch:get_resource_allocations:160 - Starting task group...
2023-05-30 15:59:33.985 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users [0]: Response (9) in 0.48s.
2023-05-30 15:59:34.369 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/eb8d73da-8c1c-7996-b085-332e777dff1a/resourceallocations/allocations [0]: Response (18) in 0.36s.
2023-05-30 15:59:34.440 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/0d100bc1-8376-7ac5-7bbb-055cdc20497d/resourceallocations/allocations [0]: Response (18) in 0.42s.
2023-05-30 15:59:34.862 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/2f365dcd-6ada-f6d8-8d73-4f393745e513/resourceallocations/allocations [0]: Response (36) in 0.42s.
2023-05-30 15:59:34.966 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/30c9d126-7cdc-53fc-3b05-7f98b25e3c98/resourceallocations/all

In [257]:
ville = "2f365dcd-6ada-f6d8-8d73-4f393745e513"

src.severa.fetch.group_sum(["user"], df[df["user"] == ville])
#df[df["user"] == ville]

,values,user
0,2023-03-02 1.415532 2023-03-03 1.415532 ...,2f365dcd-6ada-f6d8-8d73-4f393745e513


In [2]:
df

,businessunit-user,is_internal,value,user,project,date,forecast-date
0,TIE,False,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,"Nihdin Käkikello-kortteli (10669), tietomallik...",2023-05-30,2023-02-27
1,TIE,False,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,"Nihdin Käkikello-kortteli (10669), tietomallik...",2023-05-30,2023-02-28
2,TIE,False,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,"Nihdin Käkikello-kortteli (10669), tietomallik...",2023-05-30,2023-03-01
3,TIE,False,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,"Nihdin Käkikello-kortteli (10669), tietomallik...",2023-05-30,2023-03-02
4,TIE,False,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,"Nihdin Käkikello-kortteli (10669), tietomallik...",2023-05-30,2023-03-03
...,...,...,...,...,...,...,...
25,BAD,False,0.333333,af48336f-0979-b971-4927-9b09d53a3cef,"Yhtenäiskoulu, tietomallikoordinointi",2023-05-30,2023-06-26
26,BAD,False,0.333333,af48336f-0979-b971-4927-9b09d53a3cef,"Yhtenäiskoulu, tietomallikoordinointi",2023-05-30,2023-06-27
27,BAD,False,0.333333,af48336f-0979-b971-4927-9b09d53a3cef,"Yhtenäiskoulu, tietomallikoordinointi",2023-05-30,2023-06-28
28,BAD,False,0.333333,af48336f-0979-b971-4927-9b09d53a3cef,"Yhtenäiskoulu, tietomallikoordinointi",2023-05-30,2023-06-29


In [259]:
src.severa.fetch.group_sum(["user"], df[df["user"] == ville]).loc[0, "values"].loc["2023-07-01":"2023-07-30"].sum()

11.927868852459017

In [338]:
from datetime import timedelta
from datetime import date

date(2023, 5, 29) + timedelta(days=3)

datetime.date(2023, 6, 1)

In [227]:
# Quick mock data
a = pd.Series(1, index=pd.date_range("2023-01-01", "2023-01-05", freq="D"))
b = pd.Series(1, index=pd.date_range("2023-01-03", "2023-01-08", freq="D"))
c = pd.Series(1, index=pd.date_range("2023-01-04", "2023-01-12", freq="D"))

df = pd.DataFrame({"user": ["A", "B", "B"], "unit": ["TIE"]*3, "values":[a,b,c]})

# Add series with datetime indices together
def combine_series(df):
    return reduce(partial(pd.Series.add,fill_value=0), df)

# This works nicely...
#ok = combine_series(df["values"])

# ...outputs a DF, indices are OK:
# 2023-01-01    1.0
# 2023-01-02    1.0
# 2023-01-03    2.0
# 2023-01-04    3.0
# ...
# 2023-01-12    1.0
# Freq: D, dtype: float64

# This 'works' in that the values are summed correctly,
# but the indices are gone now
#not_ok = df.groupby(["unit", "user"], as_index=False).agg(combine_series)

# This is seemingly a np array
#not_ok.loc["B", "values"]
# array([1., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

grouping = ["unit", "user"]
grouped = df.groupby(grouping)

pd.DataFrame({"values":combine_series(v["values"])} | dict(zip(grouping, k)) for k, v in grouped)



,values,unit,user
0,2023-01-01 1 2023-01-02 1 2023-01-03 ...,TIE,A
1,2023-01-03 1.0 2023-01-04 2.0 2023-01-05...,TIE,B


In [298]:


hours_per_day = 1.5
index=pd.date_range(
    "2023-05-01",
    "2023-07-30",
    freq="D",
)
pd.DataFrame(index=index, data={"date":"2023-05-20", "user": "ville","value":hours_per_day})[]

,date,user,value
2023-05-01,2023-05-20,ville,1.5
2023-05-02,2023-05-20,ville,1.5
2023-05-03,2023-05-20,ville,1.5
2023-05-04,2023-05-20,ville,1.5
2023-05-05,2023-05-20,ville,1.5
...,...,...,...
2023-07-26,2023-05-20,ville,1.5
2023-07-27,2023-05-20,ville,1.5
2023-07-28,2023-05-20,ville,1.5
2023-07-29,2023-05-20,ville,1.5
